# Set up Data Frames

In [2]:
import numpy as np
import pandas as pd

In [42]:
# Dataset 1 is R&D data from Kaggle
ip = pd.read_csv("data/Top 2000 RD Companies.csv", sep=",")
ip.rename(columns={"Top 2000 R&D rank":"rank", "Company name":"company", "Company's headquarter location":"hq_country", "Industry":"industry", "R&D expenditures Share":"rdspend_share", "Patents Share":"patent_share", "Trademarks Share":"trademark_share"}, inplace=True)

# Dataset 2 is population data which will be used for per cap calculations
pop = pd.read_csv("data/population.csv", sep=",", usecols = ["Country", "Population"])
pop.rename(columns={"Country":"hq_country", "Population":"pop"}, inplace = True)

In [43]:
ip.head(5)

,rank,company,hq_country,industry,rdspend_share,patent_share,trademark_share
0,1,ALPHABET,United States,IT services,2.269,0.553,0.431
1,2,SAMSUNG ELECTRONICS,Korea,Computers & electronics,1.842,3.634,1.157
2,3,MICROSOFT,United States,Publishing & broadcasting,1.831,0.358,0.243
3,4,VOLKSWAGEN,Germany,Transport equipment,1.694,0.848,0.423
4,5,HUAWEI INVESTEMENT & HOLDING CO,China,Computers & electronics,1.582,1.703,0.801


In [44]:
pop.head(5)

,hq_country,pop
0,Argentina,1000
1,Australia,1000
2,Austria,1000
3,Belgium,1000
4,Brazil,1000


In [45]:
# Merge datasets

ip_pop = pd.merge(left=ip, right=pop, on="hq_country", how="left")



In [46]:
ip_pop.head(5)

,rank,company,hq_country,industry,rdspend_share,patent_share,trademark_share,pop
0,1,ALPHABET,United States,IT services,2.269,0.553,0.431,1000
1,2,SAMSUNG ELECTRONICS,Korea,Computers & electronics,1.842,3.634,1.157,1000
2,3,MICROSOFT,United States,Publishing & broadcasting,1.831,0.358,0.243,1000
3,4,VOLKSWAGEN,Germany,Transport equipment,1.694,0.848,0.423,1000
4,5,HUAWEI INVESTEMENT & HOLDING CO,China,Computers & electronics,1.582,1.703,0.801,1000


# Analysis 1A.  Which countries host the greatest number of companies in the Top 2000 R&D ranks?

In [51]:
country_group = ip_pop.groupby("hq_country")

for key, values in country_group:
    print(key)
    print("-"*70)
    print(values)
    print("\n")

Argentina
----------------------------------------------------------------------
      rank       company hq_country                  industry  rdspend_share  \
1334  1335  DESPEGAR.COM  Argentina  Admin & support services          0.009   

      patent_share  trademark_share   pop  
1334           0.0              0.0  1000  


Australia
----------------------------------------------------------------------
      rank                         company hq_country  \
204    205                             CSL  Australia   
217    218                         TELSTRA  Australia   
287    288         NATIONAL AUSTRALIA BANK  Australia   
469    470              ARISTOCRAT LEISURE  Australia   
555    556  COMMONWEALTH BANK OF AUSTRALIA  Australia   
932    933                             AMP  Australia   
949    950                        COCHLEAR  Australia   
1436  1437                          NUFARM  Australia   
1446  1447                   COMPUTERSHARE  Australia   
1748  1749       

In [55]:
country_group.agg(['count'])['company']

,count
hq_country,
Argentina,1
Australia,11
Austria,13
Belgium,11
Brazil,6
Canada,21
China,365
Chinese Taipei,64
Denmark,25


# Analysis 1B.  On a per capita basis?